In [ ]:
import tensorflow as tf
import gradio as gr
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.layers import TextVectorization

with open(r'E:\New folder (2)\PROJECT\New folder (2)\data.pkl', 'rb') as f:
    data = pickle.load(f)
    X = data['X']

columns=['id', 'comment_text', 'Toxic', 'Severe_toxic', 'Obscene', 'Threat','Insult', 'Identity_hate']

MAX_FEATURES= 200000

vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

ab= vectorizer.adapt(X.values)



model = tf.keras.models.load_model(r'toxicity.keras')

In [ ]:
from deep_translator import GoogleTranslator

def translatetoenglish(text):
    
    #hindi_text = GoogleTranslator(source='english', target='hindi').translate(text)
    hindi_text = GoogleTranslator(source='hindi', target='english').translate(text)

    return hindi_text

In [ ]:
def score_comment(comment):
    commen = translatetoenglish(comment)
    vectorized_comment = vectorizer([commen])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment '),
                        outputs='text')

interface.launch(share=True)